In [24]:
from WorkforceSentimentMonitoring.preprocessing import preprocessing
from WorkforceSentimentMonitoring.data import get_data, merge, get_prepaired_data
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [25]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...


TypeError: get_data() missing 1 required positional argument: 'path_to_data'

In [19]:
submission, train, test = get_data("/Users/theresasporn/code/theresa-sporn/workforce_sentiment_monitoring/raw_data")

In [20]:
text = merge(submission, train, test)
text

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4,5,5,4,5,5,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2,3,3,5,3,5,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,4,5,5,4,5,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2,5,5,4,5,4,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5,5,5,5,5,4,Execellent for engineers Impact driven. Best t...
6,Review of Google,"The people are great to work with, good perks.",Wide variety in experience based on team.,NaN,4,5,5,5,4,3,Review of Google The people are great to work ...
8,Great Company With No Down Sides,Really fun work environment with startup,Really no down sides that I saw.,NaN,5,5,5,5,5,5,Great Company With No Down Sides Really fun wo...
10,Accountant,I like and learned from the beginning accounting,High clerical work in the Sane record,Management and Finance must be close relation ...,5,5,5,5,5,4,Accountant I like and learned from the beginni...
11,Great culture,Very inspiring engineers to work with.,"Easy to get lost, as it is such a big company.",NaN,3,5,4,5,3,3,Great culture Very inspiring engineers to work...
12,Great place to work,The people are awesome. Lots of perks. Reduce ...,There's sometimes too wide a variety of things...,NaN,4,5,5,5,5,3,Great place to work The people are awesome. Lo...


In [21]:
from WorkforceSentimentMonitoring.encoders import Preprocessor
import sklearn

In [22]:
preprocessor = Preprocessor()

In [23]:
preprocessor.fit_transform(text)

AttributeError: ("'list' object has no attribute 'split'", 'occurred at index summary')